# 05 - ETL: Infraestructura Petrolera (SPATIAL JOIN)

**Objetivo:** Procesar datos de infraestructura petrolera y asignarlos a parroquias mediante análisis espacial

**Estrategia:** SPATIAL JOIN usando coordenadas geográficas (shapefiles)

**Entradas:**
- Shapefiles Petróleo: `Pozos.shp` y `v_aa011_contaminacion_hidrocarburo_pMPoint.shp`
- Shapefile Parroquias: `nxparroquia.shp` (CONALI)
- CSV procesado: `parroquias_con_salud.csv` (del Notebook 04)

**Salidas:**
- CSV: `infraestructura_petrolera_procesada.csv` (infraestructura con código de parroquia)
- CSV: `parroquias_con_petroleo.csv` (tabla agregada por parroquia)

---

## 1. Setup e Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Rutas
BASE_DIR = Path('../data')
RAW_PETROLEO = BASE_DIR / 'raw' / 'maate_petroleo'
RAW_CONALI = BASE_DIR / 'raw' / 'conali_limites'  # ← RUTA CORREGIDA
PROCESSED_DIR = BASE_DIR / 'processed'

print("✅ Librerías cargadas")
print(f"📁 Directorio petróleo: {RAW_PETROLEO}")
print(f"📁 Directorio CONALI: {RAW_CONALI}")
print(f"📁 Directorio processed: {PROCESSED_DIR}")

# Verificar directorios
for dir_path, name in [(RAW_PETROLEO, 'Petróleo'), (RAW_CONALI, 'CONALI')]:
    if dir_path.exists():
        print(f"✅ Directorio {name} encontrado")
    else:
        print(f"⚠️ Directorio {name} NO encontrado")

✅ Librerías cargadas
📁 Directorio petróleo: ../data/raw/maate_petroleo
📁 Directorio CONALI: ../data/raw/conali_limites
📁 Directorio processed: ../data/processed
✅ Directorio Petróleo encontrado
✅ Directorio CONALI encontrado


---
## 2. Cargar Shapefiles de Petróleo

In [2]:
print("🗺️ CARGANDO SHAPEFILES DE PETRÓLEO\n")
print("="*70)

# Cargar pozos
pozos = gpd.read_file(RAW_PETROLEO / 'Pozos.shp')
print(f"✅ Pozos cargados: {len(pozos):,} registros")
print(f"   CRS: {pozos.crs}")

# Cargar sitios contaminados
contaminacion = gpd.read_file(RAW_PETROLEO / 'v_aa011_contaminacion_hidrocarburo_pMPoint.shp')
print(f"✅ Sitios contaminados cargados: {len(contaminacion):,} registros")
print(f"   CRS: {contaminacion.crs}")

# Cargar otras infraestructuras (opcional)
gasoductos = gpd.read_file(RAW_PETROLEO / 'Gasoductos_Petroecuador.shp')
poliductos = gpd.read_file(RAW_PETROLEO / 'Poliductos_Petroecuador.shp')
refinerias = gpd.read_file(RAW_PETROLEO / 'Refinerias2.shp')

print(f"✅ Gasoductos cargados: {len(gasoductos)} registros")
print(f"✅ Poliductos cargados: {len(poliductos)} registros")
print(f"✅ Refinerías cargadas: {len(refinerias)} registros")

print("\n" + "="*70)

🗺️ CARGANDO SHAPEFILES DE PETRÓLEO

✅ Pozos cargados: 6,287 registros
   CRS: EPSG:4326
✅ Sitios contaminados cargados: 7,850 registros
   CRS: EPSG:32717
✅ Gasoductos cargados: 19 registros
✅ Poliductos cargados: 10 registros
✅ Refinerías cargadas: 3 registros



---
## 3. Explorar Estructura de Pozos

In [3]:
print("📋 ESTRUCTURA DE POZOS:\n")
print(f"Columnas: {list(pozos.columns)}")
print(f"\n📊 Primeros 5 registros:")
print(pozos.head())

print(f"\n📍 Información geométrica:")
print(f"   Tipo de geometría: {pozos.geometry.type.unique()}")
print(f"   CRS: {pozos.crs}")
print(f"   Bounds: {pozos.total_bounds}")

📋 ESTRUCTURA DE POZOS:

Columnas: ['nombre', 'cuenca', 'provincia', 'bloq', 'campo', 'operad', 'x', 'y', 'st_coord', 'datum', 'official_n', 'original_o', 'well_class', 'well_geome', 'geometry']

📊 Primeros 5 registros:
             nombre   cuenca  provincia  bloq         campo  \
0  Johanna Este-019  Oriente  Sucumbios  B062  Johanna Este   
1  Drago Norte-D055  Oriente  Sucumbios  B057   Drago Norte   
2         Auca-Q164  Oriente   Orellana  B061          Auca   
3  Johanna Este-023  Oriente  Sucumbios  B062  Johanna Este   
4     Anaconda-A010  Oriente   Orellana  B061      Anaconda   

                         operad          x           y   st_coord    datum  \
0  Andes Petroleum Ecuador Ltd.  333670.14  9986292.97  UTM18 Sur  PSAD 56   
1              Petroamazonas EP  305275.25  9983034.13  UTM18 Sur  PSAD 56   
2              Petroamazonas EP  290251.50  9917768.51  UTM18 Sur  PSAD 56   
3  Andes Petroleum Ecuador Ltd.  334723.25  9990189.81  UTM18 Sur  PSAD 56   
4           

---
## 4. Explorar Estructura de Contaminación

In [4]:
print("📋 ESTRUCTURA DE SITIOS CONTAMINADOS:\n")
print(f"Columnas: {list(contaminacion.columns)}")
print(f"\n📊 Primeros 5 registros:")
print(contaminacion.head())

print(f"\n📍 Información geométrica:")
print(f"   Tipo de geometría: {contaminacion.geometry.type.unique()}")
print(f"   CRS: {contaminacion.crs}")
print(f"   Bounds: {contaminacion.total_bounds}")

📋 ESTRUCTURA DE SITIOS CONTAMINADOS:

Columnas: ['codigo_de_', 'nombre', 'bloque_pet', 'anio_de_re', 'coordenada', 'tipo_fuent', 'coordenad0', 'estado', 'anio_elimi', 'area_rehab', 'operadora', 'texto_asoc', 'nombre_ter', 'geometry']

📊 Primeros 5 registros:
  codigo_de_             nombre bloque_pet  anio_de_re     coordenada  \
0      AA011  TERMINAL BEATERIO       None           0  773878.000000   
1      AA011               None  BLOQUE 57           0  983972.488824   
2      AA011               None  BLOQUE 57           0  983972.488824   
3      AA011               None  BLOQUE 57           0  984572.135169   
4      AA011               None  BLOQUE 57           0  984633.044999   

  tipo_fuent    coordenad0        estado anio_elimi  area_rehab operadora  \
0    PISCINA  9.964097e+06  NO REMEDIADO       2017      2057.0      None   
1    PISCINA  9.971217e+06     REMEDIADO          0         0.0      None   
2    PISCINA  9.971217e+06     REMEDIADO          0         0.0      No

---
## 5. Cargar Shapefile de Parroquias (CONALI)

In [5]:
print("🗺️ CARGANDO SHAPEFILE DE PARROQUIAS (CONALI)\n")
print("="*70)

# Cargar parroquias
parroquias_shp = gpd.read_file(RAW_CONALI / 'LIMITE_PARROQUIAL_CONALI_CNE_2022.shp')
print(f"✅ Parroquias cargadas: {len(parroquias_shp):,} registros")
print(f"   CRS: {parroquias_shp.crs}")

print(f"\n📋 Columnas disponibles:")
print(list(parroquias_shp.columns))

print(f"\n📊 Primeras 3 parroquias:")
print(parroquias_shp[['CODPAR', 'PROVINCIA', 'CANTON', 'PARROQUIA', 'geometry']].head(3))

print("\n" + "="*70)

🗺️ CARGANDO SHAPEFILE DE PARROQUIAS (CONALI)

✅ Parroquias cargadas: 1,236 registros
   CRS: EPSG:32717

📋 Columnas disponibles:
['OBJECTID', 'CODPRO', 'PROVINCIA', 'CODCAN', 'CANTON', 'CODPAR', 'PARROQUIA', 'ESTADO', 'Shape_Leng', 'Shape_Area', 'geometry']

📊 Primeras 3 parroquias:
  CODPAR PROVINCIA  CANTON        PARROQUIA  \
0   0285     AZUAY  CUENCA            BAÑOS   
1   0730     AZUAY  CUENCA            CUMBE   
2   0845     AZUAY  CUENCA  CHAUCHA / ANGAS   

                                            geometry  
0  POLYGON ((714383.483 9679807.792, 714390.309 9...  
1  POLYGON ((719475.594 9663878.887, 719478.891 9...  
2  POLYGON ((685768.602 9685990.681, 685768.699 9...  



---
## 6. Reprojectar CRS (Sistema de Coordenadas)

**Importante:** Todos los GeoDataFrames deben tener el mismo CRS para hacer spatial join

In [6]:
print("🔄 REPROYECTANDO CRS\n")
print("="*70)

# CRS objetivo: usar el de las parroquias (típicamente EPSG:4326 - WGS84)
target_crs = parroquias_shp.crs
print(f"CRS objetivo: {target_crs}")

# Reprojectar pozos
print(f"\nPozos:")
print(f"  CRS original: {pozos.crs}")
pozos_reproj = pozos.to_crs(target_crs)
print(f"  CRS reproyectado: {pozos_reproj.crs}")
print(f"  ✅ Pozos reproyectados")

# Reprojectar contaminación
print(f"\nContaminación:")
print(f"  CRS original: {contaminacion.crs}")
contaminacion_reproj = contaminacion.to_crs(target_crs)
print(f"  CRS reproyectado: {contaminacion_reproj.crs}")
print(f"  ✅ Contaminación reproyectada")

print("\n" + "="*70)
print("✅ TODOS LOS DATOS EN EL MISMO CRS")

🔄 REPROYECTANDO CRS

CRS objetivo: EPSG:32717

Pozos:
  CRS original: EPSG:4326
  CRS reproyectado: EPSG:32717
  ✅ Pozos reproyectados

Contaminación:
  CRS original: EPSG:32717
  CRS reproyectado: EPSG:32717
  ✅ Contaminación reproyectada

✅ TODOS LOS DATOS EN EL MISMO CRS


---
## 7. SPATIAL JOIN: Pozos con Parroquias

Asignar cada pozo a su parroquia usando el predicado espacial `within` (dentro de)

In [7]:
print("🎯 SPATIAL JOIN: POZOS → PARROQUIAS\n")
print("="*70)

# Spatial join: pozos dentro de parroquias
pozos_con_parroquia = gpd.sjoin(
    pozos_reproj,
    parroquias_shp[['CODPAR', 'PROVINCIA', 'CANTON', 'PARROQUIA', 'geometry']],
    how='left',
    predicate='within'
)

# Revisar resultados
print(f"✅ Spatial join completado")
print(f"\n📊 Estadísticas:")
print(f"   Total pozos: {len(pozos_reproj):,}")
print(f"   Pozos asignados a parroquia: {pozos_con_parroquia['CODPAR'].notna().sum():,}")
print(f"   Pozos NO asignados: {pozos_con_parroquia['CODPAR'].isna().sum():,}")

# Calcular tasa de asignación
tasa_asignacion = (pozos_con_parroquia['CODPAR'].notna().sum() / len(pozos_reproj)) * 100
print(f"   Tasa de asignación: {tasa_asignacion:.1f}%")

print(f"\n📋 Columnas resultantes:")
print(list(pozos_con_parroquia.columns))

print("\n" + "="*70)

🎯 SPATIAL JOIN: POZOS → PARROQUIAS

✅ Spatial join completado

📊 Estadísticas:
   Total pozos: 6,287
   Pozos asignados a parroquia: 6,250
   Pozos NO asignados: 37
   Tasa de asignación: 99.4%

📋 Columnas resultantes:
['nombre', 'cuenca', 'provincia', 'bloq', 'campo', 'operad', 'x', 'y', 'st_coord', 'datum', 'official_n', 'original_o', 'well_class', 'well_geome', 'geometry', 'index_right', 'CODPAR', 'PROVINCIA', 'CANTON', 'PARROQUIA']



---
## 8. SPATIAL JOIN: Contaminación con Parroquias

In [8]:
print("🎯 SPATIAL JOIN: CONTAMINACIÓN → PARROQUIAS\n")
print("="*70)

# Spatial join: sitios contaminados dentro de parroquias
contaminacion_con_parroquia = gpd.sjoin(
    contaminacion_reproj,
    parroquias_shp[['CODPAR', 'PROVINCIA', 'CANTON', 'PARROQUIA', 'geometry']],
    how='left',
    predicate='within'
)

# Revisar resultados
print(f"✅ Spatial join completado")
print(f"\n📊 Estadísticas:")
print(f"   Total sitios contaminados: {len(contaminacion_reproj):,}")
print(f"   Sitios asignados a parroquia: {contaminacion_con_parroquia['CODPAR'].notna().sum():,}")
print(f"   Sitios NO asignados: {contaminacion_con_parroquia['CODPAR'].isna().sum():,}")

# Calcular tasa de asignación
tasa_asignacion = (contaminacion_con_parroquia['CODPAR'].notna().sum() / len(contaminacion_reproj)) * 100
print(f"   Tasa de asignación: {tasa_asignacion:.1f}%")

print("\n" + "="*70)

🎯 SPATIAL JOIN: CONTAMINACIÓN → PARROQUIAS

✅ Spatial join completado

📊 Estadísticas:
   Total sitios contaminados: 7,850
   Sitios asignados a parroquia: 7,842
   Sitios NO asignados: 8
   Tasa de asignación: 99.9%



---
## 9. Consolidar Infraestructura Petrolera

Combinar pozos y sitios contaminados en una sola tabla

In [9]:
print("🔗 CONSOLIDANDO INFRAESTRUCTURA PETROLERA\n")
print("="*70)

# Crear columna de tipo de infraestructura
pozos_con_parroquia['tipo_infraestructura'] = 'Pozo'
contaminacion_con_parroquia['tipo_infraestructura'] = 'Sitio Contaminado'

# Seleccionar columnas comunes
cols_comunes = ['CODPAR', 'PROVINCIA', 'CANTON', 'PARROQUIA', 'tipo_infraestructura', 'geometry']

# Concatenar ambos GeoDataFrames
infraestructura_total = pd.concat([
    pozos_con_parroquia[cols_comunes],
    contaminacion_con_parroquia[cols_comunes]
], ignore_index=True)

print(f"✅ Infraestructura consolidada")
print(f"\n📊 Total registros: {len(infraestructura_total):,}")
print(f"\nDistribución por tipo:")
print(infraestructura_total['tipo_infraestructura'].value_counts())

# Filtrar solo los que tienen parroquia asignada
infraestructura_asignada = infraestructura_total[infraestructura_total['CODPAR'].notna()].copy()
print(f"\n📍 Infraestructura con parroquia asignada: {len(infraestructura_asignada):,}")

print("\n" + "="*70)

🔗 CONSOLIDANDO INFRAESTRUCTURA PETROLERA

✅ Infraestructura consolidada

📊 Total registros: 14,137

Distribución por tipo:
tipo_infraestructura
Sitio Contaminado    7850
Pozo                 6287
Name: count, dtype: int64

📍 Infraestructura con parroquia asignada: 14,092



---
## 10. Agregar por Parroquia

Contar infraestructura petrolera por parroquia

In [10]:
print("📊 AGREGANDO INFRAESTRUCTURA POR PARROQUIA\n")
print("="*70)

# Contar total de infraestructura por parroquia
infraestructura_por_parroquia = infraestructura_asignada.groupby('CODPAR').size().reset_index(name='num_infraestructura_petrolera')

# Contar por tipo
pozos_por_parroquia = infraestructura_asignada[infraestructura_asignada['tipo_infraestructura'] == 'Pozo'].groupby('CODPAR').size().reset_index(name='num_pozos')
contaminacion_por_parroquia = infraestructura_asignada[infraestructura_asignada['tipo_infraestructura'] == 'Sitio Contaminado'].groupby('CODPAR').size().reset_index(name='num_sitios_contaminados')

# Merge de conteos
infraestructura_por_parroquia = infraestructura_por_parroquia.merge(
    pozos_por_parroquia, on='CODPAR', how='left'
).merge(
    contaminacion_por_parroquia, on='CODPAR', how='left'
)

# Rellenar NaN con 0
infraestructura_por_parroquia['num_pozos'].fillna(0, inplace=True)
infraestructura_por_parroquia['num_sitios_contaminados'].fillna(0, inplace=True)
infraestructura_por_parroquia['num_pozos'] = infraestructura_por_parroquia['num_pozos'].astype(int)
infraestructura_por_parroquia['num_sitios_contaminados'] = infraestructura_por_parroquia['num_sitios_contaminados'].astype(int)

print(f"✅ Agregación completada")
print(f"\n📊 Parroquias con infraestructura petrolera: {len(infraestructura_por_parroquia)}")
print(f"\n📈 Estadísticas:")
print(infraestructura_por_parroquia[['num_infraestructura_petrolera', 'num_pozos', 'num_sitios_contaminados']].describe())

print(f"\n🔝 TOP 10 PARROQUIAS CON MÁS INFRAESTRUCTURA:")
top_10 = infraestructura_por_parroquia.nlargest(10, 'num_infraestructura_petrolera')
print(top_10)

print("\n" + "="*70)

📊 AGREGANDO INFRAESTRUCTURA POR PARROQUIA

✅ Agregación completada

📊 Parroquias con infraestructura petrolera: 126

📈 Estadísticas:
       num_infraestructura_petrolera    num_pozos  num_sitios_contaminados
count                      126.00000   126.000000               126.000000
mean                       111.84127    49.603175                62.238095
std                        263.25718   151.397177               177.079166
min                          1.00000     0.000000                 0.000000
25%                          2.00000     1.000000                 0.000000
50%                          7.50000     3.000000                 2.000000
75%                         70.25000    39.750000                31.500000
max                       1511.00000  1363.000000              1062.000000

🔝 TOP 10 PARROQUIAS CON MÁS INFRAESTRUCTURA:
    CODPAR  num_infraestructura_petrolera  num_pozos  num_sitios_contaminados
30    1300                           1511       1363                

---
## 11. Cargar Tabla Base (Etnia + Salud)

In [11]:
print("📂 CARGANDO TABLA BASE (Etnia + Salud)\n")
print("="*70)

# Cargar parroquias con etnia y salud
parroquias_salud = pd.read_csv(PROCESSED_DIR / 'parroquias_con_salud.csv')

print(f"✅ Tabla base cargada: {len(parroquias_salud)} parroquias")
print(f"\n📋 Columnas disponibles:")
print(list(parroquias_salud.columns))

# Verificar que existe codigo_dpa
if 'codigo_dpa' in parroquias_salud.columns:
    print(f"\n✅ Columna 'codigo_dpa' encontrada para hacer merge")
else:
    print(f"\n⚠️ No se encuentra 'codigo_dpa', columnas disponibles: {list(parroquias_salud.columns)}")

print("\n" + "="*70)

📂 CARGANDO TABLA BASE (Etnia + Salud)

✅ Tabla base cargada: 1236 parroquias

📋 Columnas disponibles:
['codigo_dpa', 'nombre_provincia', 'nombre_canton', 'nombre_parroquia', 'centroide_lon', 'centroide_lat', 'area_km2', 'poblacion_total', 'poblacion_afro', 'pct_poblacion_afro', 'num_establecimientos', 'densidad_establecimientos_km2', 'establecimientos_por_10k_hab']

✅ Columna 'codigo_dpa' encontrada para hacer merge



---
## 12. Integrar Petróleo con Tabla Base

Merge de infraestructura petrolera con la tabla que tiene etnia + salud

In [12]:
print("🔗 INTEGRANDO PETRÓLEO CON TABLA BASE\n")
print("="*70)

# Renombrar CODPAR a codigo_dpa para hacer merge
infraestructura_por_parroquia.rename(columns={'CODPAR': 'codigo_dpa'}, inplace=True)

# CRÍTICO: Convertir codigo_dpa a string en ambas tablas
infraestructura_por_parroquia['codigo_dpa'] = infraestructura_por_parroquia['codigo_dpa'].astype(str)
parroquias_salud['codigo_dpa'] = parroquias_salud['codigo_dpa'].astype(str)

print(f"🔍 Verificando tipos de datos:")
print(f"   parroquias_salud['codigo_dpa']: {parroquias_salud['codigo_dpa'].dtype}")
print(f"   infraestructura_por_parroquia['codigo_dpa']: {infraestructura_por_parroquia['codigo_dpa'].dtype}")

# Merge con tabla base
parroquias_completas = parroquias_salud.merge(
    infraestructura_por_parroquia,
    on='codigo_dpa',
    how='left'
)

# Rellenar NaN con 0 (parroquias sin infraestructura petrolera)
cols_petroleo = ['num_infraestructura_petrolera', 'num_pozos', 'num_sitios_contaminados']
for col in cols_petroleo:
    parroquias_completas[col].fillna(0, inplace=True)
    parroquias_completas[col] = parroquias_completas[col].astype(int)

# Crear variable binaria (tiene/no tiene petróleo)
parroquias_completas['tiene_petroleo'] = (parroquias_completas['num_infraestructura_petrolera'] > 0).astype(int)

print(f"\n✅ Integración completada")
print(f"\n📊 RESUMEN:")
print(f"   Total parroquias: {len(parroquias_completas)}")
print(f"   Con infraestructura petrolera: {(parroquias_completas['tiene_petroleo'] == 1).sum()}")
print(f"   Sin infraestructura petrolera: {(parroquias_completas['tiene_petroleo'] == 0).sum()}")
print(f"\n   Total pozos: {parroquias_completas['num_pozos'].sum():,}")
print(f"   Total sitios contaminados: {parroquias_completas['num_sitios_contaminados'].sum():,}")
print(f"   Total infraestructura: {parroquias_completas['num_infraestructura_petrolera'].sum():,}")

print("\n" + "="*70)

🔗 INTEGRANDO PETRÓLEO CON TABLA BASE

🔍 Verificando tipos de datos:
   parroquias_salud['codigo_dpa']: object
   infraestructura_por_parroquia['codigo_dpa']: object

✅ Integración completada

📊 RESUMEN:
   Total parroquias: 1236
   Con infraestructura petrolera: 106
   Sin infraestructura petrolera: 1130

   Total pozos: 4,716
   Total sitios contaminados: 7,026
   Total infraestructura: 11,742



---
## 13. Calcular Densidad de Infraestructura

Densidad = infraestructura / área (km²)

In [13]:
print("📊 CALCULANDO DENSIDAD DE INFRAESTRUCTURA\n")
print("="*70)

# Verificar que existe la columna area_km2
if 'area_km2' in parroquias_completas.columns:
    # Calcular densidad
    parroquias_completas['densidad_petroleo_km2'] = (
        parroquias_completas['num_infraestructura_petrolera'] / parroquias_completas['area_km2']
    ).round(4)
    
    print("✅ Densidad calculada")
    print(f"\n📊 Estadísticas de densidad:")
    print(parroquias_completas['densidad_petroleo_km2'].describe())
    
    print(f"\n🔝 TOP 10 PARROQUIAS CON MAYOR DENSIDAD:")
    top_densidad = parroquias_completas.nlargest(10, 'densidad_petroleo_km2')[
        ['nombre_provincia', 'nombre_canton', 'nombre_parroquia', 
         'num_infraestructura_petrolera', 'area_km2', 'densidad_petroleo_km2']
    ]
    print(top_densidad.to_string())
else:
    print("⚠️ No se encontró la columna 'area_km2'. Saltando cálculo de densidad.")

print("\n" + "="*70)

📊 CALCULANDO DENSIDAD DE INFRAESTRUCTURA

✅ Densidad calculada

📊 Estadísticas de densidad:
count    1236.000000
mean        0.075093
std         0.862476
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        22.977500
Name: densidad_petroleo_km2, dtype: float64

🔝 TOP 10 PARROQUIAS CON MAYOR DENSIDAD:
     nombre_provincia       nombre_canton         nombre_parroquia  num_infraestructura_petrolera  area_km2  densidad_petroleo_km2
849       SANTA ELENA         SANTA ELENA        SAN JOSE DE ANCON                           1511     65.76                22.9775
858       SANTA ELENA             SALINAS   CARLOS ESPINOZA LARREA                             36      2.88                12.5000
854       SANTA ELENA         LA LIBERTAD              LA LIBERTAD                            264     25.28                10.4430
851       SANTA ELENA             SALINAS  JOSE LUIS TAMAYO / MUEY                            157     29.07                 5.4008


---
## 14. Validaciones

In [14]:
print("✅ VALIDACIONES FINALES\n")
print("="*70)

try:
    # 1. Total registros
    assert len(parroquias_completas) > 1200, f"Debe haber >1200 parroquias, hay {len(parroquias_completas)}"
    print(f"✅ 1. Total registros correcto ({len(parroquias_completas)})")
    
    # 2. No hay valores negativos
    assert (parroquias_completas['num_infraestructura_petrolera'] >= 0).all(), "No debe haber valores negativos"
    print("✅ 2. No hay valores negativos en infraestructura")
    
    # 3. Variable binaria correcta
    assert parroquias_completas['tiene_petroleo'].isin([0, 1]).all(), "Variable binaria debe ser 0 o 1"
    print("✅ 3. Variable binaria correcta")
    
    # 4. Coherencia binaria
    coherencia = ((parroquias_completas['num_infraestructura_petrolera'] > 0) == 
                  (parroquias_completas['tiene_petroleo'] == 1)).all()
    assert coherencia, "Variable binaria debe ser coherente con conteo"
    print("✅ 4. Variable binaria coherente con conteo")
    
    # 5. Suma de pozos y contaminación = total
    suma_check = (parroquias_completas['num_pozos'] + parroquias_completas['num_sitios_contaminados'] == 
                  parroquias_completas['num_infraestructura_petrolera']).all()
    assert suma_check, "Suma de pozos + contaminación debe = total infraestructura"
    print("✅ 5. Suma de componentes es coherente")
    
    print("\n🎉 Todas las validaciones pasaron exitosamente")
    
except AssertionError as e:
    print(f"\n❌ Error en validación: {e}")

print("\n" + "="*70)

✅ VALIDACIONES FINALES

✅ 1. Total registros correcto (1236)
✅ 2. No hay valores negativos en infraestructura
✅ 3. Variable binaria correcta
✅ 4. Variable binaria coherente con conteo
✅ 5. Suma de componentes es coherente

🎉 Todas las validaciones pasaron exitosamente



---
## 15. Exportar Datos Procesados

In [15]:
print("💾 EXPORTANDO DATOS PROCESADOS\n")
print("="*70)

# 1. Infraestructura con parroquia asignada (detallado)
output_infraestructura = PROCESSED_DIR / 'infraestructura_petrolera_procesada.csv'
infraestructura_asignada_export = infraestructura_asignada.drop(columns=['geometry'])  # Remover geometría para CSV
infraestructura_asignada_export.to_csv(output_infraestructura, index=False, encoding='utf-8')
print(f"✅ 1. Infraestructura detallada: {output_infraestructura.name}")
print(f"      Registros: {len(infraestructura_asignada_export):,}")

# 2. Tabla final integrada (Etnia + Salud + Petróleo)
output_final = PROCESSED_DIR / 'parroquias_con_petroleo.csv'
parroquias_completas.to_csv(output_final, index=False, encoding='utf-8')
print(f"\n✅ 2. Tabla final integrada: {output_final.name}")
print(f"      Parroquias: {len(parroquias_completas):,}")

print("\n🎉 EXPORTACIÓN COMPLETADA")
print("\n" + "="*70)

💾 EXPORTANDO DATOS PROCESADOS

✅ 1. Infraestructura detallada: infraestructura_petrolera_procesada.csv
      Registros: 14,092

✅ 2. Tabla final integrada: parroquias_con_petroleo.csv
      Parroquias: 1,236

🎉 EXPORTACIÓN COMPLETADA



---
## 16. Resumen Final

In [16]:
print("\n" + "="*70)
print("📊 RESUMEN DEL ETL - INFRAESTRUCTURA PETROLERA (SPATIAL JOIN)")
print("="*70)

print("\n✅ TAREAS COMPLETADAS:")
print("  1. Shapefiles de petróleo cargados (pozos + contaminación)")
print("  2. Shapefile de parroquias CONALI cargado")
print("  3. CRS reproyectado (todos al mismo sistema de coordenadas)")
print("  4. SPATIAL JOIN ejecutado (pozos → parroquias)")
print("  5. SPATIAL JOIN ejecutado (contaminación → parroquias)")
print("  6. Infraestructura consolidada y agregada por parroquia")
print("  7. Integración con tabla base (Etnia + Salud + Petróleo)")
print("  8. Variables calculadas (conteos, binaria, densidad)")
print("  9. Validaciones exitosas")
print(" 10. Datos exportados")

print("\n📊 ESTADÍSTICAS FINALES:")
print(f"  • Total parroquias: {len(parroquias_completas):,}")
print(f"  • Parroquias con petróleo: {(parroquias_completas['tiene_petroleo'] == 1).sum():,} ({(parroquias_completas['tiene_petroleo'] == 1).sum()/len(parroquias_completas)*100:.1f}%)")
print(f"  • Total infraestructura asignada: {parroquias_completas['num_infraestructura_petrolera'].sum():,}")
print(f"    - Pozos: {parroquias_completas['num_pozos'].sum():,}")
print(f"    - Sitios contaminados: {parroquias_completas['num_sitios_contaminados'].sum():,}")
print(f"  • Promedio infraestructura/parroquia (con petróleo): {parroquias_completas[parroquias_completas['tiene_petroleo']==1]['num_infraestructura_petrolera'].mean():.2f}")

print("\n📁 ARCHIVOS GENERADOS:")
print(f"  • {output_infraestructura.name} (infraestructura detallada)")
print(f"  • {output_final.name} (TABLA FINAL INTEGRADA)")

print("\n🎉 TABLA FINAL INTEGRADA:")
print("  Incluye: Etnia + Salud + Petróleo por parroquia")
print("  Variables clave de petróleo:")
print("    - num_infraestructura_petrolera (total)")
print("    - num_pozos")
print("    - num_sitios_contaminados")
print("    - tiene_petroleo (binaria 0/1)")
print("    - densidad_petroleo_km2")

print("\n🚀 PRÓXIMO PASO:")
print("  Notebook 06: Análisis Espacial y Clustering")
print("  • Clustering espacial (K-means / DBSCAN)")
print("  • Mapas coropléticos")
print("  • Análisis de correlaciones")
print("  • Identificación de hotspots")

print("\n" + "="*70)


📊 RESUMEN DEL ETL - INFRAESTRUCTURA PETROLERA (SPATIAL JOIN)

✅ TAREAS COMPLETADAS:
  1. Shapefiles de petróleo cargados (pozos + contaminación)
  2. Shapefile de parroquias CONALI cargado
  3. CRS reproyectado (todos al mismo sistema de coordenadas)
  4. SPATIAL JOIN ejecutado (pozos → parroquias)
  5. SPATIAL JOIN ejecutado (contaminación → parroquias)
  6. Infraestructura consolidada y agregada por parroquia
  7. Integración con tabla base (Etnia + Salud + Petróleo)
  8. Variables calculadas (conteos, binaria, densidad)
  9. Validaciones exitosas
 10. Datos exportados

📊 ESTADÍSTICAS FINALES:
  • Total parroquias: 1,236
  • Parroquias con petróleo: 106 (8.6%)
  • Total infraestructura asignada: 11,742
    - Pozos: 4,716
    - Sitios contaminados: 7,026
  • Promedio infraestructura/parroquia (con petróleo): 110.77

📁 ARCHIVOS GENERADOS:
  • infraestructura_petrolera_procesada.csv (infraestructura detallada)
  • parroquias_con_petroleo.csv (TABLA FINAL INTEGRADA)

🎉 TABLA FINAL INTEG